In [239]:
import pandas as pd 
import numpy as np 
import collections as cols

In [240]:
# keep this to 1 for outer metrices calculation else make it 0
outer = 1

In [241]:
qcount = 1008
if outer == 1:
    qstart = 0
    qlast = 1008
    start = 0
    last = qcount*3
    i = 1
else:
    qstart = 1008
    qlast = 1245
    start = qcount*3
    last = 3735
    qcount = 237
    i = 1009

In [242]:
df2 = pd.read_csv("Data/nitika.csv")[start:last]
df3 = pd.read_csv("Data/prerna.csv")[start:last]

In [243]:
# convert the to binary matrix 

def get_data(acolumn,right_ans):
    acolumn =  np.reshape(acolumn.values,(qcount,3))
    repd = []
    for vec in acolumn: # vec = [2,3,1]
        rep = []
        for ind in vec:
            if ind in right_ans:
                rep.append(1)
            else:
                rep.append(0)
        repd.append(rep)
    return np.array(repd)

In [244]:
mat_bin = get_data(df3["Ans"],[1,2])
mat_bin[:5]

array([[0, 0, 0],
       [1, 1, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [245]:
def precision_at_k(data):
    """
    param : data
    2-D array representing right ans in each row
    """
    prec = []
    for vec in data:
        rev_doc = 0
        tot_doc = 0
        prec_vec = []
        for doc in vec:
            if doc == 1:
                rev_doc +=1
            tot_doc +=1
            prec_vec.append(rev_doc/tot_doc)
        prec.append(np.array(prec_vec))
    return np.array(prec)

In [246]:
precision_at_k(mat_bin)[:5]

array([[0.        , 0.        , 0.        ],
       [1.        , 1.        , 0.66666667],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ]])

In [247]:
def precision_at_3(data):
    """
    param : data
    2-D array representing right ans in each row
    """
    prec_at_k = precision_at_k(mat_bin)
    k = 3-1
    prec_at_3 = [vec[k] for vec in prec_at_k]
    return np.array(prec_at_3)

In [248]:
precision_at_3(mat_bin)[:5]

array([0.        , 0.66666667, 0.        , 0.        , 0.        ])

In [249]:
def avg_precision_at_3(data):
    """
    param : data
    2-D array representing right ans in each row
    """
    data = precision_at_k(data)
    aprec = []
    for vec in data:
        aprec.append(sum(vec)/3)
    return np.array(aprec)

In [250]:
avg_precision_at_3(mat_bin)[:5]

array([0.        , 0.88888889, 0.        , 0.        , 0.        ])

In [251]:
def reciprocal_rank(data):
    """
    param : data
    2-D array representing right ans in each row
    """
    rr = []
    for vec in data: #  vec = [0,1,1]
        i = 1
        for ind in vec:
            if ind == 1:
                break
            i+=1
        #print(i)
        if i !=4:
            rr.append(1/i)
        else:
            rr.append(0)
    return np.array(rr)


In [252]:
reciprocal_rank(mat_bin)[:5]

array([0., 1., 0., 0., 0.])

In [253]:
def success_ratio(data):
    """
    param : data
    2-D array representing right ans in each row
    """
    sr_mat = []
    for vec in data:
        sr_mat.append(sum(vec)/len(vec))
    
    return np.array(sr_mat)

In [254]:
success_ratio(mat_bin)[:5]

array([0.        , 0.66666667, 0.        , 0.        , 0.        ])

In [255]:
def invert_rel(element):
    rmap = {1:2,2:1,3:0}
    return rmap[element] 

In [256]:
def ndcg(acolumn):
    """
    param : acolumn
    pandas series representing ans in each cell
    """
    idcg_vec = np.array([1/np.log2(2) , 1/np.log2(3) , 1/np.log2(4)])
    acolumn = acolumn.apply(invert_rel)
    acolumn =  np.reshape(acolumn.values,(-1,3))

    ndcg_vec = []

    for vec in acolumn: #  vec = [1,2,1]   
        idcg = sum(idcg_vec*sorted(vec,reverse=True))
        if idcg!=0:
            ndcg_vec.append(np.sum(vec*idcg_vec)/idcg)
        else:
            ndcg_vec.append(0)
    return np.array(ndcg_vec)

In [257]:
ndcg(df3["Ans"])[:5]

array([0., 1., 0., 0., 0.])

In [258]:
#get model number
def get_commom_entries(df1,df2):
    cmn = df1.Ans==df2.Ans
    entries = np.apply_along_axis(all,1,np.reshape(cmn.values,(qcount,3)))


    data = pd.read_csv("final-Ques_rand.csv")[qstart:qlast]
    df = data[["Page","Model_name"]]
    models = df.Model_name.unique()
    df["Model_num"] = df.Model_name.replace(models,[0,1,2])
    df = df[entries]

    grp = {}
    dgrp = df.groupby(df.Model_num)
    grp["Grp1_Pg"] = dgrp.get_group(0).Page.values-i
    grp["Grp2_Pg"] = dgrp.get_group(1).Page.values-i
    grp["Grp3_Pg"] = dgrp.get_group(2).Page.values-i

    return grp

In [259]:
def grp_division(rows,df2,df3):
    mod_num = get_commom_entries(df2,df3)
    grp1 = rows[mod_num["Grp1_Pg"]]
    grp2 = rows[mod_num["Grp2_Pg"]]
    grp3 = rows[mod_num["Grp3_Pg"]]
    return grp1,grp2,grp3

In [260]:
def grp_mean(grp_mat):
    return (np.mean(grp_mat[0]),np.mean(grp_mat[1]),np.mean(grp_mat[2]))

In [261]:
def calc_metrics(df2,df3):
    """
    param : acolumn
    pandas series representing ans in each cell
    """
    acolumn = df3["Ans"] # we can also add df2 as in last step we will take common
    data = get_data(acolumn,[1,2])
    m_mat = {}

    m_mat["map"] = grp_mean(grp_division(avg_precision_at_3(data),df2,df3))
    m_mat["mrr"] = grp_mean(grp_division(reciprocal_rank(data),df2,df3))
    m_mat["avg_sr"] = grp_mean(grp_division(success_ratio(data),df2,df3))
    m_mat["avg_ndcg"] = grp_mean(grp_division(ndcg(acolumn),df2,df3))
    m_mat["avg_p3"] = grp_mean(grp_division(precision_at_3(data),df2,df3))
    return m_mat

In [262]:
calc_metrics(df2,df3)

{'map': (0.045151893466500205, 0.11183891314895682, 0.098903836571998),
 'mrr': (0.06741573033707865, 0.15356622998544395, 0.14275037369207771),
 'avg_sr': (0.036204744069912614, 0.09315866084425034, 0.0881913303437967),
 'avg_ndcg': (0.0709492127469439, 0.15977172018401356, 0.15185273808332828),
 'avg_p3': (0.036204744069912614, 0.09315866084425034, 0.0881913303437967)}

In [263]:
#For starting till 1008*3
pd.DataFrame(calc_metrics(df2,df3),index=['गंगा', 'काली', 'रेनू'])

,map,mrr,avg_sr,avg_ndcg,avg_p3
गंगा,0.045152,0.067416,0.036205,0.070949,0.036205
काली,0.111839,0.153566,0.093159,0.159772,0.093159
रेनू,0.098904,0.142750,0.088191,0.151853,0.088191
